In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install --upgrade google-cloud-bigquery

In [ ]:
!pip install  google-generativeai pandas openpyxl plotly

In [ ]:
!pip install --upgrade --force-reinstall flask-sqlalchemy sqlalchemy

In [ ]:
pip install 'pyspark-ai[all]@git+https://github.com/asl3/pyspark-ai.git@add-all-dependency-group'

In [ ]:
!pip install "pyspark-ai[all]"


In [ ]:
!pip install google-cloud-aiplatform
!pip install langchain

In [ ]:
!pip install --upgrade langchain-google-vertexai

## Check Spark kernel connection

In [ ]:
!pip install matplotlib

In [ ]:
sc

In [ ]:
SparkSession

In [ ]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

### Check if it is able to connect to Hive

In [ ]:
mafmetrics = spark.sql("SELECT * FROM schema.table LIMIT 50")


In [ ]:
pandas_mafmetrics = mafmetrics.toPandas()

# pandas_datehour_totals.set_index('datehour', inplace=True)
pandas_mafmetrics.head()

In [ ]:
import warnings
warnings.filterwarnings('error', category=DeprecationWarning)

In [ ]:
import sys,os,os.path
#Just to avoid the warnings
os.environ["OPENAI_API_KEY"] = "junk"

In [ ]:
import vertexai

PROJECT_ID = ""  # @param {type:"string"}
REGION = ""  # @param {type:"string"}

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=REGION)

In [ ]:
# Utils
from langchain.schema import HumanMessage, SystemMessage
# from langchain.llms import VertexAI
from langchain_google_vertexai import VertexAI
from langchain.embeddings import VertexAIEmbeddings
# from langchain.chat_models import ChatVertexAI
from langchain_google_vertexai import ChatVertexAI
from google.cloud import aiplatform
import time
from typing import List


# LangChain
import langchain
from pydantic import BaseModel

from pyspark.sql.functions import expr


In [ ]:
print(f"LangChain version: {langchain.__version__}")

# Vertex AI

print(f"Vertex AI SDK version: {aiplatform.__version__}")

### Initialization 

In [ ]:
modelgardenchatmodel1 = ChatVertexAI(
    model_name="chat-bison@001", max_output_tokens=1000, temperature=0.2
)

In [ ]:
modelgardenchatmodel2 = ChatVertexAI(
    model_name="chat-bison@002", max_output_tokens=1000, temperature=0.2
)

In [ ]:
modelgardenchatmodel2 = ChatVertexAI(
    model_name="codechat-bison@002", max_output_tokens=1000, temperature=0.2
)

In [ ]:
from pyspark_ai import SparkAI

spark_ai = SparkAI(llm=modelgardenchatmodel1,vector_store_dir="temp/", verbose=True)
spark_ai.activate()  # active partial functions for Spark DataFrame

In [ ]:
agentresponse = mafmetrics.ai.explain()

In [ ]:
agentresponse

In [ ]:
mafmetrics.ai.transform("what all type are available?").show()


In [ ]:
mafmetrics.ai.transform("what is total count of Distros with residential filter?").show()

In [ ]:
mafmetrics.ai.transform("which type has highest count").show()


In [ ]:
mafmetrics2 = mafmetrics.ai.transform("get data by type and count")


In [ ]:
mafmetrics2.ai.plot()


In [ ]:
mafmetrics2.head()

In [ ]:
mafmetrics2.ai.plot()

In [ ]:
import plotly.express as px
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd


# Aggregate the data
df_agg = mafmetrics.groupBy("type").agg(F.sum("count").alias("total_count"))

# Convert the Spark DataFrame to a pandas DataFrame
df_pd = df_agg.toPandas()

# Create a pie chart
fig = px.pie(df_pd, values="total_count", names="type", title="Count by Type")

# Display the plot
fig.show()

In [ ]:
df_agg = mafmetrics.groupBy("type").agg(F.sum("count").alias("total_count"))
df_pd = df_agg.toPandas()

In [ ]:
df_agg.show()

In [ ]:
fig = px.pie(df_pd, values="total_count", names="type", title="Count by Type")


### Only for udf we use chat-bison@001

In [ ]:
from pyspark_ai import SparkAI

spark_ai = SparkAI(llm=chat1,verbose=True)
spark_ai.activate() 

### Now lets create udf 

In [ ]:
@spark_ai.udf
def full_name(first_name: str, last_name: str) -> str:
    """combine first name and last name.
       If field is missing replace it by None. All imports should be inside function.
    """
    ...

### test the udf

In [ ]:
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, udf

spark.udf.register("full_name", full_name, returnType=StringType())
results = (
    cnbdata
    .withColumn("first_name", col("first_name"))
    .withColumn("last_name", col("last_name"))
    .withColumn("full_name", expr("full_name(first_name, last_name)"))
)

In [ ]:
results.head()

### Transform Accuracy Improvement: Vector Similarity Search